<a href="https://colab.research.google.com/github/Storm00212/Electromagnetics/blob/main/Maxwell's_equations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is a simulation of Maxwell's equations using python code to visualize and hence better understand the concepts

In [1]:
# setup of parameters for this simulation
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib.animation import FuncAnimation

# Simulation Parameters
size = 200          # Number of spatial points
steps = 400         # Number of time steps
c0 = 1.0            # Normalized speed of light. Will adjust for future realistic sims
dx = 1.0            # Spatial step
dt = dx / (2 * c0)  # Time step (Courant stability condition)

# Initialize Fields
# E and B are shifted by half a grid point (Yee Grid)
E = np.zeros(size)
B = np.zeros(size)


# Faraday's law

In [2]:
# update function
def update_B(E, B):
    # B[i] = B[i] - dt/dx * (E[i+1] - E[i])
    B[:-1] -= (dt / dx) * (E[1:] - E[:-1])
    return B


# Ampere Maxwell's law

In [3]:
def update_E(E, B):
    # E[i] = E[i] - dt/dx * (B[i] - B[i-1])
    E[1:] -= (dt / dx) * (B[1:] - B[:-1])
    return E


# Figure for animation

In [20]:
# Create the figure for animation
fig, ax = plt.subplots(figsize=(10, 5))
line_e, = ax.plot(E, label='Electric Field (E)', color='blue')
line_b, = ax.plot(B, label='Magnetic Field (B)', color='red')
ax.set_ylim(-1.5, 1.5)
ax.legend()
ax.set_title("1D Maxwell Wave Propagation")

def animate(frame):
    global E, B

    # 1. Update B-field (Faraday's Law)
    B = update_B(E, B)

    # 2. Update E-field (Ampere-Maxwell Law)
    E = update_E(E, B)

    # 3. Source Injection (Gaussian pulse at center) for fun
    pulse = np.exp(-0.5 * ((frame - 30) / 10)**2)
    E[size // 2] += pulse

    # 4. Update plot lines
    line_e.set_ydata(E)
    line_b.set_ydata(B)
    return line_e, line_b

# Run the animation
ani = FuncAnimation(fig, animate, frames=steps, interval=30, blit=True)
plt.close() # Prevents duplicate static plot in
# Display animation in Jupyter (as HTML5 video)
HTML(ani.to_html5_video())

# 2D Format

In [22]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Grid setup for 2D visualization
x = np.linspace(-1, 1, 20)
y = np.linspace(-1, 1, 20)
X, Y = np.meshgrid(x, y)
r = np.sqrt(X**2 + Y**2)
theta = np.arctan2(Y, X)

# Parameters
R = 0.5  # Radius of the region with uniform B
dBdt = 1.0  # Rate of change of B (constant for simplicity)

# Function to compute induced E and B at time t
def get_fields(t):
    Bz = dBdt * t  # Increasing B_z over time

    # Bz array (uniform inside R, zero outside)
    Bz_array = np.zeros_like(r)
    Bz_array[r < R] = Bz

    # Induced E_phi (azimuthal component) from Faraday's law
    E_phi = np.zeros_like(r)
    inside = r < R
    E_phi[inside] = - (r[inside] / 2) * dBdt
    outside = r >= R
    E_phi[outside] = - (R**2 / (2 * r[outside])) * dBdt

    # Convert to Cartesian Ex, Ey
    Ex = -E_phi * np.sin(theta)
    Ey = E_phi * np.cos(theta)

    return Ex, Ey, Bz_array, Bz

# Set up the figure
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_title('Induced E from Changing B (Faraday\'s Law)')
ax.set_xlabel('x')
ax.set_ylabel('y')

# Color map for Bz (out of page)
pcm = ax.pcolormesh(X, Y, np.zeros_like(X), cmap='coolwarm', shading='auto', vmin=-10, vmax=10)
plt.colorbar(pcm, ax=ax, label='B_z (increasing over time)')

# Quiver for E field vectors
quiver = ax.quiver(X, Y, np.zeros_like(X), np.zeros_like(Y), color='black', scale=20)

# Time text
time_text = ax.text(0.05, 0.95, '', transform=ax.transAxes)

# Update function for animation
def update(frame):
    t = frame / 10.0  # Scale time for slower animation
    Ex, Ey, Bz_array, Bz = get_fields(t)

    # Update Bz color map
    pcm.set_array(Bz_array.ravel())

    # Update E quiver
    quiver.set_UVC(Ex, Ey)

    # Update time text
    time_text.set_text(f'Time: {t:.1f}, B_z = {Bz:.1f}')

    return pcm, quiver, time_text

# Create animation
ani = FuncAnimation(fig, update, frames=100, interval=50, blit=False)
plt.close()  # Prevents duplicate static plot in Jupyter
# Display animation in Jupyter (as HTML5 video)
HTML(ani.to_html5_video())